In [1]:
import sys
import numpy as np
import pandas as pd

In [2]:
sys.path.append("../..")
from preprocess.load_data.data_loader import load_hotel_reserve
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

In [3]:
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [11]:
reserve_tb.describe()

,people_num,total_price
count,4030.000000,4030.000000
mean,2.542184,103065.955335
std,1.120925,110288.484355
min,1.000000,3500.000000
25%,2.000000,32400.000000
50%,3.000000,64800.000000
75%,4.000000,129600.000000
max,4.000000,897600.000000


In [18]:
price_list = []
max_hindo = reserve_tb['people_num'].max()
for i in range(1, max_hindo+1):
    tmp_list = reserve_tb[reserve_tb['people_num']>=i]['total_price'].values.tolist()
    price_list.extend(tmp_list)

In [ ]:
reserve_tb

In [19]:
len(price_list)

10245

## 3-1 データ数，種類数の算出

Not Awesome

In [3]:
# groupby関数でreserve_idを集約単位に指定し、size関数でデータ数をカウント
# groupby関数の集約処理によって行番号（index）がとびとびになっているので、
# reset_index関数によって、集約単位に指定したhotel_idを集約した状態から列名に戻し、
# 新たな行名を現在の行番号を直す
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()

In [5]:
rsv_cnt_tb.head()

,hotel_id,0
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [6]:
# 集約結果の列名を設定
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']

In [8]:
rsv_cnt_tb.head()

,hotel_id,rsv_cnt
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [12]:
# groupbyでhotel_idを集約単位に指定し、
# customer_idの値をnunique関数することで顧客数をカウント
cus_cnt_tb = \
  reserve_tb.groupby('hotel_id')['customer_id'].nunique().reset_index()

In [14]:
cus_cnt_tb.head()

,hotel_id,customer_id
0,h_1,10
1,h_10,3
2,h_100,19
3,h_101,17
4,h_102,13


In [15]:
# 集約結果の列名を設定
cus_cnt_tb.columns = ['hotel_id', 'cus_cnt']

In [17]:
# merge関数を用いて、hotel_idを結合キーとして結合(「第4章 結合」で解説)
NA3_1 = pd.merge(rsv_cnt_tb, cus_cnt_tb, on='hotel_id')

In [18]:
NA3_1.head()

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


### 3-1 Awesome

In [19]:
# agg関数を利用して、集約処理をまとめて指定
# reserve_idを対象にcount関数を適用
# customer_idを対象にnunique関数を適用
result = reserve_tb \
  .groupby('hotel_id') \
  .agg({'reserve_id': 'count', 'customer_id': 'nunique'})

In [21]:
result.head()

,reserve_id,customer_id
hotel_id,,
h_1,10,10
h_10,3,3
h_100,20,19
h_101,17,17
h_102,13,13


In [22]:
# reset_index関数によって、列番号を振り直す（inplace=Trueなので、直接resultを更新）
result.reset_index(inplace=True)

In [23]:
result.head()

,hotel_id,reserve_id,customer_id
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


In [24]:
result.columns = ['hotel_id', 'rsv_cnt', 'cus_cnt']

In [25]:
result.head()

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


## 3-2　合計値の算出

### 3-2 Awesome

In [26]:
# 集約単位をhotel_idとpeople_numの組み合わせを指定
# 集約したデータからtotal_priceを取り出し、sum関数に適用することで売上合計金額を算出
result = reserve_tb \
  .groupby(['hotel_id', 'people_num'])['total_price'] \
  .sum().reset_index()

In [28]:
result.head()

,hotel_id,people_num,total_price
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200


In [29]:
# 売上合計金額の列名がtotal_priceになっているので、price_sumに変更
result.rename(columns={'total_price': 'price_sum'}, inplace=True)

In [30]:
result.head()

,hotel_id,people_num,price_sum
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200


## 3-3　極値，代表値の算出

### 3-3 Awesome

In [32]:
# total_priceを対象にmax/min/mean/median関数を適用
# Pythonのラムダ式をagg関数の集約処理に指定
# ラムダ式にはnumpy.percentileを指定しパーセントタイル値を算出（パーセントは20指定）
result = reserve_tb \
  .groupby('hotel_id') \
  .agg({'total_price': ['max', 'min', 'mean', 'median',
                        lambda x: np.percentile(x, q=20)]}) \
  .reset_index()
result.columns = ['hotel_id', 'price_max', 'price_min', 'price_mean',
                  'price_median', 'price_20per']

In [33]:
result.head()

,hotel_id,price_max,price_min,price_mean,price_median,price_20per
0,h_1,208800,26100,112230.000000,104400,73080
1,h_10,67200,11200,42933.333333,50400,26880
2,h_100,57600,4800,27600.000000,28800,9600
3,h_101,168000,14000,75764.705882,56000,30800
4,h_102,72000,12000,32769.230769,24000,18000


## 3-4　ばらつき具合の算出

### 3-4 Awesome

In [34]:
# total_priceに対して、var関数とstd関数を適用し、分散値と標準偏差値を算出
result = reserve_tb \
  .groupby('hotel_id') \
  .agg({'total_price': ['var', 'std']}).reset_index()
result.columns = ['hotel_id', 'price_var', 'price_std']

# データ数が1件だったときは、分散値と標準偏差値がnaになっているので、0に置き換え
result.fillna(0, inplace=True)

In [36]:
result.head()

,hotel_id,price_var,price_std
0,h_1,3.186549e+09,56449.526127
1,h_10,8.258133e+08,28736.968061
2,h_100,3.198316e+08,17883.835689
3,h_101,2.402441e+09,49014.703676
4,h_102,3.576923e+08,18912.755159


## 3-5　最頻値の算出

### 3-5 Awesome

In [37]:
# round関数で四捨五入した後に、mode関数で最頻値を算出
reserve_tb['total_price'].round(-3).mode()

0    10000
1    20000
2    40000
dtype: int64

## 3-6a 順位の算出

### 3-6a Awesome

In [38]:
# rank関数で並び替えるために、データ型を文字列からtimestamp型に変換
# （「第10章 日時型」で解説）
reserve_tb['reserve_datetime'] = pd.to_datetime(
  reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S'
)

In [40]:
# log_noを新たな列として追加
# 集約単位の指定はgroup_byを利用
# 顧客ごとにまとめたreserve_datetimeを生成し、rank関数によって順位を生成
# ascendingをTrueにすることで昇順に設定(Falseだと降順に設定)
reserve_tb['log_no'] = reserve_tb \
  .groupby('customer_id')['reserve_datetime'] \
  .rank(ascending=True, method='first')

In [41]:
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5.0


### 3-6b Awesome

In [42]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

In [43]:
# 予約回数を計算(「3-1 データ数、種類数の算出」の例題を参照)
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']

In [44]:
rsv_cnt_tb.head()

,hotel_id,rsv_cnt
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [45]:
# 予約回数をもとに順位を計算
# ascendingをFalseにすることで降順に指定
# methodをminに指定し、同じ値の場合は取り得る最小順位に指定
rsv_cnt_tb['rsv_cnt_rank'] = rsv_cnt_tb['rsv_cnt'] \
  .rank(ascending=False, method='min')

# 必要のないrsv_cntの列を削除
rsv_cnt_tb.drop('rsv_cnt', axis=1, inplace=True)

In [46]:
rsv_cnt_tb.head()

,hotel_id,rsv_cnt_rank
0,h_1,235.0
1,h_10,300.0
2,h_100,12.0
3,h_101,43.0
4,h_102,139.0


In [48]:
# ランキングが高い順に表示
rsv_cnt_tb.sort_values('rsv_cnt_rank', ascending=True).head()

,hotel_id,rsv_cnt_rank
158,h_241,1.0
50,h_144,2.0
231,h_37,3.0
48,h_142,3.0
281,h_82,5.0
